In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Notebook for cleaning data so that it is ready for ML.


In [ ]:
def load_dataset(path: str): 
    #load in the data as a pandas dataframe
    df = pd.read_csv(path, na_values=["Unknown","N/A"])
    return df

def check_drop_duplicates(df: pd.DataFrame):
    #check for id and age duplicates
    is_duplicate = df.duplicated(subset=["id","age"])
    total_duplicates = is_duplicate.sum()
    df = df.drop_duplicates(subset=["id","age"])
    return df
                                        

In [ ]:
def remove_missing_numeric_values(df: pd.DataFrame):
    #select numeric columns
    numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
    #only bmi has missing values (4 %)
    #remove all children
    mask = df["age"] > 17
    df = df[mask]
    #remove missing values
    df = df.dropna(subset=numeric_columns)
    return df
    
    

If we plot histograms of the continuous numerical features, we observe some unreasonable outliers for avg_glucose_level and bmi.
The likely explanation is mistakes in entering the data. 

In [ ]:
def plot_histograms(f: pd.DataFrame):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))  # define the figure and subplots
    axes = axes.ravel()  # array to 1D
    cols = ["avg_glucose_level", "bmi"]
    colors = ['tab:blue', 'tab:orange']  # list of colors for each subplot, otherwise all subplots will be one color

    for col, color, ax in zip(cols, colors, axes):
        f[col].plot(kind='box', ax=ax, color=color, label=col, title=col)
        
    fig.tight_layout()
    plt.show()
    


We will replace the outliers in these two features with the median in each case:

In [ ]:
def replace_outliers(df: pd.DataFrame):
    bmi_max = 50 #values above this will be replaced
    bmi_median = np.median(df["bmi"])
    print(bmi_median)
    avg_glucose_max = 180
    glucose_median = np.median(df["avg_glucose_level"])
    print(glucose_median)
    df["bmi"] = df["bmi"].apply(lambda x: bmi_median if x > bmi_max else x)
    df["avg_glucose_level"] = df["avg_glucose_level"].apply(lambda x: glucose_median if x > avg_glucose_max else x)
    return df

In [ ]:
def remove_missing_categorical_values(df: pd.DataFrame):
    text_columns = df.select_dtypes(include=object).columns.tolist()
    # only smoking status has missing values here, but 20 % are missing. This is too high a proportion to simply drop missing values
    # for now we will just replace with mode for this column: "never smoked"
    smoking_status = df["smoking_status"]
    replacement_dict = {np.nan : "never smoked"}
    smoking_status = smoking_status.replace(replacement_dict)
    df["smoking_status"] = smoking_status
    return df
    
    

In [ ]:
def clean_data(path: str):
    """_summary_

    Args:
        path (str): relative path to dataset

    Returns:
        dataframe: pandas dataframe of cleaned data
    """
    df = load_dataset(path)
    df = check_drop_duplicates(df)
    df = remove_missing_numeric_values(df)
    df = replace_outliers(df)
    df = remove_missing_categorical_values(df)
    return df

In [ ]:
if __name__ == "__main__":
    data_path = "healthcare-dataset-stroke-data.csv"
    cleaned_data = clean_data(path=data_path)
    #saves cleaned dataframe as a pickle file in workspace
    plot_histograms(cleaned_data)
    cleaned_data.to_pickle("cleaned_data.pkl")
